In [1]:
import pandas as pd
import numpy as np

#### movieLens 100K 데이터 불러오기
* u.user : 사용자 데이터
* u.item : 영화에 대한 데이터
    * movie_id : 영화 id
    * title : 제목 
    * release date : 개봉날짜
    * 'unknown' 부터 'Western'까지 19가지의 변수로 0또는 1로 장르를 표현
        * 예를들어 액션 영화는 'Action'에 1로 표시되며 나머지 18개 장르 변수는 0으로 표시
* u.data : 영화평가(rating) 데이터
    * user_id : 사용자 id
    * movie_id : 영화 id
    * rating : 평점 (1~5)
    * timestamp : 평가한 연도/날짜/시간

In [2]:
# u.user(사용자 데이터) 파일을 DataFrame으로 읽기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('./Data/u.user', sep='|', names=u_cols, encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [3]:
# u.item(영화에 대한 데이터) 파일을 DataFrame으로 읽기
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('./Data/u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
# u.data(평화평가(rating) 데이터) 파일을 DataFrame으로 읽기
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./Data/u.data', sep='\t', names=r_cols, encoding='latin-1') 
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


#### 인기제품 방식
개별 자용자에 대한 정보가 없는 경우나, 정확도에 관계없이 가장 간단한 추천을 제공해야 하는 상황에서 사용할 수 있는 방법은 모든 사람에게 똑같은 추천을 하는 것이다. 당연히 모든 사람들에게 가장 인기 있는 제품(best-seller)을 추천 하는 것이 가장 합리적일 것이다. 인기 있는 제품을 추천하기 위해서는 각 제품(영화)에 대한 평가(평점)를 평균해서 평균값이 가장 높은 것을 순서대로 추천하면 된다.

In [5]:
# Best-seller 추천 
def recom_movie1(n_items):
    # 각 영화의 평점평균인 movie_mean을 내림차순 정렬후 그중 상휘부터 지정된 개수(n_item)만큼을 movie_sort에 저장
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
    # movies에 있는 영화 중에서 선정된 n_item개의 영화에 대한 정보를 저장
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

#각 영화(movie_id)별 평점(rating)의 평균을 구해 movie_mean에 저장
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
recom_movie1(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

#### 정확도 계산
예측이 정확할 수록 실제값과 예측값 사이의 차이가 줄어들기 때문에 RMSE(Root Mean Squared Error)가 작을수록 정확한 추천 시스템이라고 할 수 있다.  
RMSE를 수식으로 표시하면 아래와 같다.
$$RMSE = \sqrt{\frac{1}{N}\sum_{i=1}^{N}(y_i-\widehat{y_i})^2}$$

In [6]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

rmse = []
# rating에 있는 user_id를 다 추출해서 한 user씩 확인한다.
for user in set(ratings.index):
    # 각 사용자가 평가한 모든 영화의 평점을 y_ture에 저장한다. 
    y_true = ratings.loc[user]['rating']
    # 해당 사용자가 평가한 영화의 평점평균을 y_pred에 저장한다.
    # 현재 사용하는 추천 알고리즘이 best-seller 방식이기 때문에 평점 평균이 해당 영화의 예측값이라고 할 수 있다.
    y_pred = movie_mean[ratings.loc[user]['movie_id']]
    # RMSE를 구한다.
    error = RMSE(y_true, y_pred)
    rmse.append(error)
print(np.mean(rmse))

0.996007224010567
